In [1]:
import os
import json
import pickle
import numpy as np
import pandas as pd

# DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/data'
# VAL_TO_IDX_DATA_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Server/data'

# MODEL_PATH = '/opt/ml/final-project-level3-recsys-05/Model/Model-Experiment/model'

In [8]:
!pip install --upgrade pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/4e/dd/4a77fb4cb7d207fbeb77dfc7c022131d295767504eabb5836fcd63b644a1/pandas-2.1.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 10.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


In [20]:
problem_df = pd.read_csv('/Users/im_jungwoo/Desktop/project/Google Extension/server/models/solvedacAPI/result/data.csv')

problem_df.columns.values[1] = 'problem_id'
problem_df.columns.values[14] = 'problem_tags'

problem_df['problem_id'] = problem_df['problem_id'].astype(str)

In [21]:
problem_df.head()

Unnamed: 0 problem_id  titleKo  \
0           0       1000      A+B   
1           1       1001      A-B   
2           2       1002       터렛   
3           3       1003  피보나치 함수   
4           4       1004    어린 왕자   

                                              titles  isSolvable  isPartial  \
0  [{'language': 'en', 'languageDisplayName': 'en...        True      False   
1  [{'language': 'ko', 'languageDisplayName': 'ko...        True      False   
2  [{'language': 'ko', 'languageDisplayName': 'ko...        True      False   
3  [{'language': 'ko', 'languageDisplayName': 'ko...        True      False   
4  [{'language': 'ko', 'languageDisplayName': 'ko...        True      False   

   acceptedUserCount  level  votedUserCount  sprout  givesNoRating  \
0             265818      1             140    True          False   
1             226989      1              56    True          False   
2              34754      8             250   False          False   
3              48576      8             206   False          False   
4              14389      8             137   False          False   

   isLevelLocked  averageTries  official                     problem_tags  \
0           True        2.5177      True  implementation,arithmetic,math,   
1           True        1.4291      True  implementation,arithmetic,math,   
2          False        4.4631      True         case_work,geometry,math,   
3          False        3.0534      True                              dp,   
4          False        2.1833      True                   geometry,math,   

  metadata  
0       {}  
1       {}  
2       {}  
3       {}  
4       {}

In [22]:
problem_df = problem_df.loc[:, ['problem_id', 'level','problem_tags']]

In [32]:
# # problem_association_seq 제작

# import re

# problem_association_seq_list = set()
# problem_id_list = problem_df['problem_id'].tolist()

# problem_association_list = problem_df['problem_association'].tolist()
# title_list = problem_df['title'].tolist()
# for title, problem_association in zip(title_list, problem_association_list):
#     problem_association = eval(problem_association)
#     if problem_association:
#         problem_association += [title]
#         problem_association = sorted(list(set(problem_id_list) & set(re.findall(r'([0-9]+?)번', '#'.join(problem_association)))))
#         problem_association_seq_list.add(tuple(problem_association))

# problem_association_seq_list = list(map(list, problem_association_seq_list))

# len(problem_association_seq_list)

In [30]:
# level로 seq 제작

problem_level_seq_list = []

group_df = problem_df.groupby('level')

for level, df in group_df:
    problem_level_seq_list.append(df['problem_id'].tolist())

print(len(problem_level_seq_list))

31


In [24]:
# arr = '임정우,백지원,노명은,한상진,'
# arr = [arr]
# tags = eval(arr)
# tags
arr = ''
arr = arr.split(',')
arr.pop()
print(arr)

[]


In [25]:
# tag로 seq 제작

problem_tag_seq_list = []

tag_list = []

def get_preprocessing_tags(tags):
    global tag_list
    # tags = eval(tags)
    if type(tags) != 'float':
      tags = str(tags).split(',')
      tags.pop()
      tags = [tag for tag in tags]
      tag_list += tags
    return tags

problem_df['preprocessing_tags'] = problem_df['problem_tags'].apply(lambda x : get_preprocessing_tags(x))
tag_list = list(set(tag_list))
problem_num_list = problem_df['problem_id'].tolist()

tag_df = pd.DataFrame(data = np.zeros((len(problem_num_list), len(tag_list))), columns = tag_list, index = problem_num_list)

for df in problem_df.iloc:
    tag_df.loc[df['problem_id'], df['preprocessing_tags']] = 1

for tag in tag_list:
    problem_tag_seq = tag_df[tag_df[tag] == 1].index.tolist()
    problem_tag_seq_list.append(problem_tag_seq)

len(problem_tag_seq_list)

203

In [34]:
tag_df

monotone_queue_optimization  backtracking  centroid  priority_queue  \
1000                           0.0           0.0       0.0             0.0   
1001                           0.0           0.0       0.0             0.0   
1002                           0.0           0.0       0.0             0.0   
1003                           0.0           0.0       0.0             0.0   
1004                           0.0           0.0       0.0             0.0   
...                            ...           ...       ...             ...   
24996                          0.0           0.0       0.0             0.0   
24997                          0.0           0.0       0.0             0.0   
24998                          0.0           0.0       0.0             0.0   
24999                          0.0           0.0       0.0             0.0   
25000                          0.0           0.0       0.0             0.0   

       cactus  greedy  duality  combinatorics  scc  implementation  ...  \
1000      0.0     0.0      0.0            0.0  0.0             1.0  ...   
1001      0.0     0.0      0.0            0.0  0.0             1.0  ...   
1002      0.0     0.0      0.0            0.0  0.0             0.0  ...   
1003      0.0     0.0      0.0            0.0  0.0             0.0  ...   
1004      0.0     0.0      0.0            0.0  0.0             0.0  ...   
...       ...     ...      ...            ...  ...             ...  ...   
24996     0.0     0.0      0.0            0.0  0.0             0.0  ...   
24997     0.0     0.0      0.0            0.0  0.0             0.0  ...   
24998     0.0     0.0      0.0            0.0  0.0             0.0  ...   
24999     0.0     0.0      0.0            0.0  0.0             0.0  ...   
25000     0.0     0.0      0.0            0.0  0.0             0.0  ...   

       lucas  hash_set  mst  cht  generating_function   mo  knuth_x  dag  \
1000     0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
1001     0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
1002     0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
1003     0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
1004     0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
...      ...       ...  ...  ...                  ...  ...      ...  ...   
24996    0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
24997    0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
24998    0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
24999    0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   
25000    0.0       0.0  0.0  0.0                  0.0  0.0      0.0  0.0   

       palindrome_tree  burnside  
1000               0.0       0.0  
1001               0.0       0.0  
1002               0.0       0.0  
1003               0.0       0.0  
1004               0.0       0.0  
...                ...       ...  
24996              0.0       0.0  
24997              0.0       0.0  
24998              0.0       0.0  
24999              0.0       0.0  
25000              0.0       0.0  

[23427 rows x 203 columns]

In [26]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec
import datetime

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 1
        self.loss_to_be_subed = 0
        self.loss_now = 987654321

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed

        self.loss_to_be_subed = loss
        print(loss_now, self.loss_now)
        if loss_now < self.loss_now:
            self.loss_now = loss_now
            # model.save(os.path.join(MODEL_PATH, 'clean-Word2Vec-CBOW-problem_association_seq-problem_level_seq-problem_tag_seq-vs128.model'))
            print(f'Loss after epoch {self.epoch}: {loss_now}')
            # print('Model 저장')
        self.epoch += 1

start = datetime.datetime.now()

sentences = problem_level_seq_list + problem_tag_seq_list

model = Word2Vec(
                sentences = sentences,
                seed = 22,
                epochs = 300,
                min_count = 1,
                vector_size = 128,
                sg = 0,
                negative = 10,
                window = 987654321,
                compute_loss = True,
                callbacks=[callback()],
                 )

print("Time passed: " + str(datetime.datetime.now() - start))

92271.6796875 987654321
Loss after epoch 1: 92271.6796875
107227.6484375 92271.6796875
103461.328125 92271.6796875
102583.875 92271.6796875
98682.375 92271.6796875
91846.90625 92271.6796875
Loss after epoch 6: 91846.90625
86354.5 91846.90625
Loss after epoch 7: 86354.5
81304.3125 86354.5
Loss after epoch 8: 81304.3125
75998.5625 81304.3125
Loss after epoch 9: 75998.5625
71966.8125 75998.5625
Loss after epoch 10: 71966.8125
68141.625 71966.8125
Loss after epoch 11: 68141.625
64793.875 68141.625
Loss after epoch 12: 64793.875
60817.875 64793.875
Loss after epoch 13: 60817.875
58789.875 60817.875
Loss after epoch 14: 58789.875
57098.75 58789.875
Loss after epoch 15: 57098.75
55502.875 57098.75
Loss after epoch 16: 55502.875
53678.5 55502.875
Loss after epoch 17: 53678.5
52475.0 53678.5
Loss after epoch 18: 52475.0
50576.375 52475.0
Loss after epoch 19: 50576.375
49619.125 50576.375
Loss after epoch 20: 49619.125
48234.625 49619.125
Loss after epoch 21: 48234.625
47182.5 48234.625
Loss aft

In [28]:
# 유사한 단어 찾기
similar_words = model.wv.most_similar("1005")

# 결과 출력
print("Words similar to '1005':")
for word, score in similar_words:
    print(f"{word}: {score:.4f}")


Words similar to '1005':
1516: 0.9936
5847: 0.9825
18780: 0.9656
9470: 0.9075
13691: 0.9056
23906: 0.9054
14757: 0.9049
2252: 0.9031
5937: 0.9024
2623: 0.9018


In [48]:
import math
problem_id = "1005"
similar_problem = model.wv.most_similar(problem_id)
(problem, score) = similar_problem[0]
print(problem, math.floor(score * 100))

23032 99


In [29]:
model.save("word2vec_model.bin")

item2vec.ipynb     word2vec_model.bin


In [19]:
# from gensim.models import Word2Vec

# with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'clean_problem_id_to_idx.json'), 'r', encoding = 'utf-8') as f:
#     problem_id2idx = json.load(f)

# with open(os.path.join(VAL_TO_IDX_DATA_PATH, 'clean_idx_to_problem_id.json'), 'r', encoding = 'utf-8') as f:
#     idx2problem_id = json.load(f)

ids = problem_df["problem_id"].to_list()
vectors = []
for id in ids:
  vectors.append(model.wv[id])
vectors = np.array(vectors)

In [20]:
vectors

array([[-0.04551158, -0.48126405,  1.2746272 , ...,  0.9965144 ,
        -1.8930271 , -1.559648  ],
       [-0.04949784, -0.48569328,  1.2866275 , ...,  1.0147113 ,
        -1.879358  , -1.5627702 ],
       [-0.80592644,  1.5268614 ,  0.45178264, ...,  1.4711316 ,
        -1.4122283 , -1.491984  ],
       ...,
       [-1.0900366 ,  0.15229999,  0.38579842, ...,  0.96586955,
        -0.54003   , -0.07141001],
       [-0.81841326,  1.5171108 ,  0.43075237, ...,  1.4584861 ,
        -1.4011346 , -1.4647173 ],
       [-1.0906614 ,  0.16902418,  0.39034867, ...,  0.951178  ,
        -0.5338964 , -0.05297158]], dtype=float32)

In [21]:
# with open(os.path.join(MODEL_PATH, 'clean-Word2Vec-CBOW-problem_association_seq-problem_level_seq-problem_tag_seq-vs128-vector.pickle'), 'wb') as file:
#     pickle.dump(vectors, file, protocol = 4)

In [22]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state = 0)
tsne_vectors = tsne.fit_transform(vectors)

/Users/im_jungwoo/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/backend/context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
[Errno 2] No such file or directory: 'sysctl'
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "/Users/im_jungwoo/anaconda3/lib/python3.11/site-packages/joblib/externals/loky/backend/context.py", line 235, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "/Users/im_jungwoo/anaconda3/lib/python3.11/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/im_jungwoo/anaconda3/lib/python3.11/subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "/Users/im_jungwoo/anaconda3/lib/python3.11/subprocess.py", line 195

In [29]:
vectors.shape

(23427, 128)

In [30]:
tsne_vectors

array([[  8.951612 , 103.69334  ],
       [  7.2406597, 104.38716  ],
       [-48.631878 , -48.149372 ],
       ...,
       [-27.117722 ,  19.793943 ],
       [-43.629272 , -48.186962 ],
       [-35.236393 ,  23.094257 ]], dtype=float32)

In [28]:
import matplotlib.pyplot as plt

plt.figure(figsize = (12, 12))
problem_df.rename(columns={"Unnamed: 0.1":"id_to_idx"},inplace=True)
for level in sorted(problem_df['level'].unique()):
    idx = problem_df[problem_df['level'] == level]['id_to_idx'].tolist()
    idx = list(map(int, idx))
    #idx = clean_idx(idx)
    plt.scatter(x = tsne_vectors[idx, 0], y = tsne_vectors[idx, 1], alpha = 0.7, label = f'{level}')

plt.legend()
plt.show()

KeyError: 'id_to_idx'

<Figure size 1200x1200 with 0 Axes>

In [26]:
tsne_vectors_df = pd.DataFrame(data = tsne_vectors, columns = ['x', 'y'])

In [27]:
idx = tsne_vectors_df[((-60 <= tsne_vectors_df['x']) & (tsne_vectors_df['x'] <= -35)) & ((-100 <= tsne_vectors_df['y']) & (tsne_vectors_df['y'] <= -50))].index.tolist()

In [33]:
problem_df

problem_id  level                     problem_tags  \
0           1000      1  implementation,arithmetic,math,   
1           1001      1  implementation,arithmetic,math,   
2           1002      8         case_work,geometry,math,   
3           1003      8                              dp,   
4           1004      8                   geometry,math,   
...          ...    ...                              ...   
23422      24996      0                              NaN   
23423      24997      0                              NaN   
23424      24998      0                              NaN   
23425      24999      8           binary_search,sorting,   
23426      25000      0                              NaN   

                       preprocessing_tags  
0      [implementation, arithmetic, math]  
1      [implementation, arithmetic, math]  
2             [case_work, geometry, math]  
3                                    [dp]  
4                        [geometry, math]  
...                                   ...  
23422                                  []  
23423                                  []  
23424                                  []  
23425            [binary_search, sorting]  
23426                                  []  

[23427 rows x 4 columns]

In [42]:
ProblemLevelDict = {}
ProblemTagsDict = {}
for i in range(len(problem_df)):
    ProblemLevelDict[problem_df.loc[i, 'problem_id']] = str(problem_df.loc[i, 'level'])
    ProblemTagsDict[problem_df.loc[i, 'problem_id']] = problem_df.loc[i, 'preprocessing_tags']

In [43]:
import json
with open('./ProblemLevelDict.json','w') as f:
  json.dump(ProblemLevelDict, f, ensure_ascii=False, indent=4)
with open('./ProblemTagsDict.json','w') as f:
  json.dump(ProblemTagsDict, f, ensure_ascii=False, indent=4)